<a href="https://colab.research.google.com/github/JeffersonEspinalA/Prueba/blob/main/Copia_de_C%C3%B3digo_fuente_Hito_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importar librerias

In [1]:
import tensorflow
import pandas as pd
import numpy as np

from keras.models import Sequential
from keras import optimizers
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras_preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD

Cargar datos

In [2]:
dataset = pd.read_csv('drive/MyDrive/HAM10000/HAM10000_metadata.csv')
dataset['image_full_name'] = dataset['image_id']+'.jpg'

Creación del modelo

In [3]:
modelo = Sequential()
modelo.add(Conv2D(64, (3, 3), activation='relu', input_shape=(32, 32, 3)))
modelo.add(MaxPooling2D(pool_size=(2, 2)))
modelo.add(Dropout(0.5))
modelo.add(Flatten())
modelo.add(Dense(128, activation='relu'))
modelo.add(Dropout(0.5))
modelo.add(Dense(7, activation='sigmoid'))

Dividir datos: entrenamiento, prueba y validación

In [4]:
X = dataset[['image_full_name','dx','lesion_id']]
Y = X.pop('dx').to_frame()
X_entre, X_prueba, Y_entre, Y_prueba = train_test_split(X, Y, test_size=0.15, random_state=42)
X_entre, X_val, Y_entre, Y_val = train_test_split(X_entre, Y_entre, test_size=0.15, random_state=42)

In [5]:
entre = pd.concat([X_entre, Y_entre],axis=1)
val = pd.concat([X_val, Y_val],axis=1)
prueba = pd.concat([X_prueba, Y_prueba],axis=1)

In [6]:
encoder = LabelEncoder()
encoder.fit(val['dx'])
tipo_indice_entre = encoder.transform(val['dx']) 
val['label'] = tipo_indice_entre

In [7]:
encoder = LabelEncoder()
encoder.fit(prueba['dx'])
tipo_indice_prueba = encoder.transform(prueba['dx']) 
prueba['label'] = tipo_indice_prueba

In [8]:
entre_generator = ImageDataGenerator(rescale=1./255, rotation_range=10, zoom_range = 0.1, 
                                     width_shift_range=0.0, height_shift_range=0.00)

In [9]:
datos_entre = entre_generator.flow_from_dataframe(dataframe=entre, x_col="image_full_name", y_col="dx",
                                                  batch_size=32, directory="drive/MyDrive/HAM10000/HAM10000_images",
                                                  shuffle=True, class_mode="categorical", target_size=(32,32))

Found 7235 validated image filenames belonging to 7 classes.


In [10]:
prueba_generator = ImageDataGenerator(rescale=1./255,rotation_range=10, zoom_range=0.1, 
                                      width_shift_range=0.0, height_shift_range=0.02)

In [11]:
datos_prueba = prueba_generator.flow_from_dataframe(dataframe=prueba, x_col="image_full_name", y_col="dx",
                                                    directory="drive/MyDrive/HAM10000/HAM10000_images",
                                                    shuffle=False, batch_size=1, class_mode=None, target_size=(32,32))

Found 1503 validated image filenames.


In [12]:
datos_val = prueba_generator.flow_from_dataframe(dataframe=val, x_col="image_full_name", y_col="dx",
                                                 directory="drive/MyDrive/HAM10000/HAM10000_images",
                                                 batch_size=64, shuffle=False, class_mode="categorical", target_size=(32,32))

Found 1277 validated image filenames belonging to 7 classes.


Compilar modelo

In [13]:
sgd = optimizers.SGD(learning_rate=0.01, clipvalue=0.5)
modelo.compile(optimizer=sgd,loss="categorical_crossentropy",metrics=["accuracy"])

Entrenar modelo

In [18]:
historial = modelo.fit_generator(generator=datos_entre, steps_per_epoch=datos_entre.samples//datos_entre.batch_size,
                               validation_data=datos_val, verbose=1, validation_steps=datos_val.samples//datos_val.batch_size,
                               epochs=25)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  This is separate from the ipykernel package so we can avoid doing imports until


Epoch 1/25
226/226 [==============================] - 127s 556ms/step - loss: 0.8269 - accuracy: 0.7007 - val_loss: 0.7986 - val_accuracy: 0.7113
Epoch 2/25
226/226 [==============================] - 127s 564ms/step - loss: 0.8255 - accuracy: 0.7012 - val_loss: 0.7894 - val_accuracy: 0.7130
Epoch 3/25
226/226 [==============================] - 125s 554ms/step - loss: 0.8056 - accuracy: 0.7083 - val_loss: 0.7772 - val_accuracy: 0.7122
Epoch 4/25
226/226 [==============================] - 132s 586ms/step - loss: 0.8112 - accuracy: 0.7043 - val_loss: 0.7789 - val_accuracy: 0.7171
Epoch 5/25
226/226 [==============================] - 130s 575ms/step - loss: 0.8069 - accuracy: 0.7060 - val_loss: 0.7807 - val_accuracy: 0.7179
Epoch 6/25
226/226 [==============================] - 128s 567ms/step - loss: 0.8103 - accuracy: 0.7080 - val_loss: 0.7947 - val_accuracy: 0.7064
Epoch 7/25
226/226 [==============================] - 133s 590ms/step - loss: 0.8048 - accuracy: 0.7072 - val_loss: 0.7775 -

Evaluación porcentaje de precisión

In [19]:
datos_prueba.reset()
predicciones = modelo.predict_generator(datos_prueba, steps=datos_prueba.samples/datos_prueba.batch_size, verbose=1)
x_pred = np.argmax(predicciones, axis=1)
cant = np.where(x_pred == tipo_indice_prueba)
print(f"Tasa de precision del modelo: {int((np.count_nonzero(cant) / len(tipo_indice_prueba)) * 100)} %")

   1/1503 [..............................] - ETA: 2:43

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  


1503/1503 [==============================] - 31s 20ms/step
Tasa de precision del modelo: 71 %


Guardar modelo

In [20]:
modelo.save('Trabajo_Final.h5')